In [ ]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import os
from google.colab import drive
import time

# Check TensorFlow version
print("TensorFlow Version:", tf.__version__)

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define paths (adjust these based on your Google Drive structure)
model_path = '/content/drive/MyDrive/Colab Notebooks/data/final_optimized_model.keras'
data_file_path = '/content/drive/MyDrive/Colab Notebooks/data/ISIC_2019.csv'  # Path to the provided data file
image_dir = '/content/drive/MyDrive/Colab Notebooks/data/ISIC_2019_Training_Input/'  # Path to the image directory
output_csv_path = '/content/drive/MyDrive/Colab Notebooks/data/per_image_predictions_isic2019.csv'  # Path to save predictions

# Load the data file
data_df = pd.read_csv(data_file_path)
print("Data loaded successfully. Number of samples:", len(data_df))

# Define class labels (matching app.py)
class_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
label_mapping = {
    'MEL': 'mel',
    'NV': 'nv',
    'BCC': 'bcc',
    'AK': 'akiec',
    'BKL': 'bkl',
    'DF': 'df',
    'VASC': 'vasc'
}

# Define the focal loss function (matching app.py, without tf.keras decorator)
def focal_loss(alpha=None, gamma=1.0):
    if alpha is None:
        alpha = [0.1347, 0.0857, 0.0401, 0.3832, 0.0396, 0.0066, 0.3102]  # From app.py
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        ce = -y_true * tf.math.log(y_pred)
        weight = tf.convert_to_tensor(alpha, dtype=tf.float32) * tf.pow(1.0 - y_pred, gamma)
        return tf.reduce_mean(weight * ce)
    return loss

# Function to preprocess a single image (matching app.py)
def preprocess_image(image_path, target_size=(300, 300)):
    #print(f"Preprocessing started for {image_path}")
    if not os.path.exists(image_path):
        raise ValueError(f"File not found: {image_path}")

    # Read and decode the image (same as app.py)
    img_raw = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img_raw, channels=3)

    # Resize the image
    img = tf.image.resize(img, target_size)

    # Apply EfficientNet preprocessing (same as app.py)
    img = tf.keras.applications.efficientnet.preprocess_input(img)

    # Debug: Print shape and min/max values
    img_np = img.numpy()
    #print("Image shape:", img_np.shape)
    #print("Min, Max:", img_np.min(), img_np.max())

    # Add batch dimension
    img_np = np.expand_dims(img_np, axis=0)
    #print("Preprocessing completed")
    return img_np

# Function to get the ground truth label
def get_ground_truth(row):
    for col in label_mapping.keys():
        if row[col] == 1:
            return label_mapping[col]
    return None

# Load the pre-trained model with the custom loss
custom_objects = {'loss': focal_loss(alpha=[0.1347, 0.0857, 0.0401, 0.3832, 0.0396, 0.0066, 0.3102], gamma=1.0)}
model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
print("Model loaded successfully.")

# List to store predictions
results = []

# Process each image one by one
for idx, row in data_df.iterrows():
    image_id = row['image']
    image_path = os.path.join(image_dir, f"{image_id}.jpg")

    if not os.path.exists(image_path):
       # print(f"Image not found: {image_path}")
        continue

    try:
        img_array = preprocess_image(image_path)
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        continue

    true_label = get_ground_truth(row)
    if true_label is None:
        print(f"No ground truth label found for {image_id}")
        continue

    start_time = time.time()
    prediction = model.predict(img_array, verbose=0)
    inference_time = time.time() - start_time

    predicted_class_idx = np.argmax(prediction[0])
    predicted_label = class_labels[predicted_class_idx]
    confidence = float(prediction[0][predicted_class_idx])

    results.append({
        'image_id': image_id,
        'true_label': true_label,
        'predicted_label': predicted_label,
        'confidence': confidence,
        'inference_time': inference_time
    })

    if (idx + 1) % 100 == 0:
        print(f"Processed {idx + 1}/{len(data_df)} images")

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv_path, index=False)
print(f"Predictions saved to {output_csv_path}")

# Calculate and display per-class accuracy
def calculate_per_class_accuracy(df):
    correct_predictions = df[df['true_label'] == df['predicted_label']]
    per_class_accuracy = correct_predictions.groupby('true_label').size() / df.groupby('true_label').size()
    return per_class_accuracy * 100

per_class_acc = calculate_per_class_accuracy(results_df)
print("\nAccuracy Rate by Skin Lesion Type (%):")
print(per_class_acc.fillna(0).round(2))

# Calculate and display overall accuracy
overall_accuracy = (results_df['true_label'] == results_df['predicted_label']).mean() * 100
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

TensorFlow Version: 2.18.0
Mounted at /content/drive
Data loaded successfully. Number of samples: 24703
Model loaded successfully.
Processed 100/24703 images
Processed 300/24703 images
Processed 400/24703 images
Processed 500/24703 images
Processed 600/24703 images
Processed 700/24703 images
Processed 800/24703 images
Processed 900/24703 images
Processed 1100/24703 images
Processed 1300/24703 images
Processed 1500/24703 images
Processed 1700/24703 images
Processed 2000/24703 images
Processed 2100/24703 images
Processed 2200/24703 images
Processed 2300/24703 images
Processed 2400/24703 images
Processed 2600/24703 images
Processed 2700/24703 images
Processed 2800/24703 images
Processed 3400/24703 images
Processed 3500/24703 images
Processed 3900/24703 images
Processed 4300/24703 images
Processed 4400/24703 images
Processed 4500/24703 images
Processed 4600/24703 images
Processed 4800/24703 images
Processed 4900/24703 images
Processed 5000/24703 images
Processed 5400/24703 images
Processed